# Шаг 6. Машинное обучение

In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

import numpy as np
import pandas as pd

In [2]:
RESULT_df_itog = pd.read_csv('RESULT_df_itog')
RESULT_df_itog.head()

,TRADEDATE,CLOSE_IMOEX,OPEN_IMOEX,HIGH_IMOEX,LOW_IMOEX,VALUE_IMOEX,CAPITALIZATION_IMOEX,CLOSE_MOEXMM,OPEN_MOEXMM,HIGH_MOEXMM,...,SMA_long,RSI_IMOEX,ATR_IMOEX,Price_GOLD_Lagged,Day_type_выходной,Day_type_рабочий день,DYNAMICS_IMOEX_Fall,DYNAMICS_IMOEX_Growth,ECONOMIC_GROWTH_Recession,ECONOMIC_GROWTH_Rise
0,2013-09-02,1367.53,1364.75,1373.73,1364.27,1.274407e+10,6.096417e+12,2109.53,2103.86,2120.72,...,1443.644725,100.0,22.518929,1487.36,False,True,False,True,False,True
1,2013-09-03,1373.82,1366.99,1384.18,1366.99,2.216425e+10,6.124461e+12,2102.51,2109.79,2123.56,...,1443.644725,100.0,22.518929,1487.36,False,True,False,True,False,True
2,2013-09-04,1375.66,1372.98,1376.42,1365.21,1.692969e+10,6.132675e+12,2109.74,2101.98,2112.03,...,1443.644725,100.0,22.518929,1487.36,False,True,False,True,False,True
3,2013-09-05,1422.40,1375.66,1422.40,1373.29,3.763559e+10,6.341025e+12,2161.35,2110.70,2162.06,...,1443.644725,100.0,22.518929,1487.36,False,True,False,True,False,True
4,2013-09-06,1423.40,1421.86,1448.75,1418.80,5.218912e+10,6.345500e+12,2157.45,2159.63,2199.92,...,1443.644725,100.0,22.518929,1487.36,False,True,False,True,False,True


## Постановка задачи и отбор признаков

Нашей задачей является **прогнозирование цен на золото**. Будем это делать с помощью моделей линейной регрессии, метода к ближайших соседей, бэггинга для регрессии. 

Зависимой (прогнозируемой) переменной (y) являются цены на золото (Price_GOLD в датафрейме)

Параметрами модели (X) являются следующие признаки:

Отбор параметров произведем с учетом корреляции признаков, полученной на этапе визуализации, независимости параметров модели и их качественного вклада в цену золота. Между IMOEX и MOEXMM корреляция с ценой золота больше у IMOEX, поэтому в качестве параметра выберем его, у курса USD RUB наивысшая корреляция с ценой золота, для учета инфляции возьмем ключевую ставку, так как ее корреляция с ценой золота также выше, а инфляция и ключевая ставка - зависимые параметры, поэтому оба параметра не должны быть включены. Для учета динамики экономического роста возьмем параметр ECONOMIC_GROWTH_Rise.

In [3]:
columns = ['AVER_IMOEX', 'USD_RUB', 'Price_GOLD','Key_rate', 'ECONOMIC_GROWTH_Rise']
RESULT_ML = RESULT_df_itog[columns]
RESULT_ML.head()

,AVER_IMOEX,USD_RUB,Price_GOLD,Key_rate,ECONOMIC_GROWTH_Rise
0,1366.140,33.2522,1487.36,8.0,True
1,1370.405,33.2522,1487.36,8.0,True
2,1374.320,33.3693,1492.60,8.0,True
3,1399.030,33.4656,1510.36,8.0,True
4,1422.630,33.3901,1494.07,8.0,True


Проверим пропуски в данных:

In [4]:
print(RESULT_ML.isnull().any().any())

False


In [5]:
X = RESULT_ML.drop('Price_GOLD', axis=1) # объявили параметры модели
y = RESULT_ML['Price_GOLD'] # объявили зависимую переменную
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) # делим выборку на train и test

## 1. Модель линейной регрессии

Одним из наиболее распространенных вариантов прогнозирования является линейная регрессия, выполним ее для нашей выборки

In [6]:
model_regression = LinearRegression()
model_regression.fit(X_train, y_train)
y_pred_regr = model_regression.predict(X_test)

In [7]:
print(f' Коэффициенты модели: \n {model_regression.coef_}')
print(f'Точка отсчета модели: \n{model_regression.intercept_}')

 Коэффициенты модели: 
 [ 5.99030728e-01  6.62635562e+01  7.34282333e+01 -6.28519634e+02]
Точка отсчета модели: 
-2593.6731218950126


In [8]:
mse_regr = mean_squared_error(y_test, y_pred_regr)
print(f' Оценка MSE: \n{mse_regr}')
mae_regr = mean_absolute_error(y_test, y_pred_regr)
print(f' Оценка MAE: \n{mae_regr}')
r2_regr = r2_score(y_test, y_pred_regr)
print(f' Оценка r^2: \n{r2_regr}')

 Оценка MSE: 
313642.78134057904
 Оценка MAE: 
446.7575779006021
 Оценка r^2: 
0.8772529368592028


* **Вывод:** Относительная оценка r^2 модели линейной регрессии показывает хорошее значение, изменение 88% данных объясняется моделью, однако средняя абсолютная и средняя квадратичная ошибка показывают плохие результаты: значения наших данных не такие большие, поэтому ошибку = 446 можно считать большой, не говоря уже о шестизначной MSE. Попробуем другую модель.

## 2. KNN для регрессии


Отмасмштабируем признаки:

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
# подбираем наилучший k, оценивать будем через метрику r^2, так как нам важно как модель объясняет изменения переменной
k = np.array([2, 3, 4, 5, 7, 10, 13, 15, 20, 25, 30, 35, 40, 45, 50, 70, 80, 90, 100, 150, 200, 250])
n = len(k)
r2_train = np.zeros(n)
r2_test = np.zeros(n)

for i in range(n):
  knn = KNeighborsRegressor(n_neighbors=k[i], p=1, n_jobs=10)
  knn.fit(X_train_scaled, y_train)
  y_pred_train = knn.predict(X_train_scaled)
  y_pred_test = knn.predict(X_test_scaled)
  r2_train[i] = r2_score(y_train, y_pred_train)
  r2_test[i] = r2_score(y_test, y_pred_test)
d = pd.DataFrame({'k': k, 'R2 Train': r2_train, 'R2 Test': r2_test})
d

,k,R2 Train,R2 Test
0,2,0.999381,0.998802
1,3,0.999144,0.998692
2,4,0.998728,0.998692
3,5,0.998373,0.998541
4,7,0.997596,0.997674
5,10,0.996022,0.996134
6,13,0.994675,0.994565
7,15,0.993884,0.993777
8,20,0.991089,0.991504
9,25,0.988492,0.989424


In [11]:
best_r2 = -1
best_k = None
for index, row in d.iterrows():
    if row['R2 Test'] > best_r2:
        best_r2 = row['R2 Test']
        best_k = row['k']

print(f"Лучшее значение k: {int(best_k)}")

Лучшее значение k: 2


In [12]:
knn = KNeighborsRegressor(n_neighbors=int(best_k), p=2, n_jobs=10)
knn.fit(X_train_scaled, y_train)

y_pred_test_final = knn.predict(X_test_scaled)
r2_test_final = r2_score(y_test, y_pred_test_final)
print(f"Лучшее значение r2:\n {r2_test_final}")

Лучшее значение r2:
 0.9986767251920006


In [19]:
mse_knn = mean_squared_error(y_test, y_pred_test_final)
print(f' Оценка MSE: \n{mse_knn}')
mae_knn = mean_absolute_error(y_test, y_pred_test_final)
print(f' Оценка MAE: \n{mae_knn}')
msle_test = mean_squared_log_error(y_test, y_pred_test_final)
print(f' Оценка MSLE на тестовой выборке: \n{round(msle_test, 4)}')
rmsle_test_knn = np.sqrt(mean_squared_log_error(y_test, y_pred_test_final))
print(f' Оценка RMSLE на тестовой выборке: \n{round(rmsle_test_knn, 4)}')

 Оценка MSE: 
3381.226243945192
 Оценка MAE: 
34.32024532861927
 Оценка MSLE на тестовой выборке: 
0.0002
 Оценка RMSLE на тестовой выборке: 
0.0154


* **Вывод:** **Оценки стали значительно лучше**: r^2 равен почти 1, значит модель практически идеально объясняет данные. MSLE равна очень маленькому числу, что говорит о том, что модель хорошо предсказывает маленькие значения. Однако MSE опять высокая, что в комбинации с низкой MSLE свидетельствует о том, что модель хорошо предсказывает маленькие значения, но плохо большие. MAE получился низкий, что подтверждает что модель неплохо предсказывает значения и не переобучена. RMLSE - аналогично имеет низкое значение

## 3. Бэггинг

Попробуем еще улучшить параметры и реализовать бэггинг для модели регрессии - уменьшим дисперсию и попытаемся избежать переобучения.

Источник: https://github.com/hse-ds/iad-intro-ds/blob/master/2023/seminars/sem13_rf/sem13_rf_solution.ipynb

In [14]:
base_tree = DecisionTreeRegressor() # Создали решающее дерево для регрессии - базовую модель 
bag = BaggingRegressor(base_tree, n_estimators=100, n_jobs=4, random_state=100) # Создали ансамбль моделей (100 деревьев) с использованием бэггинга
bag.fit(X_train, y_train) # Обучили бэггинг 
y_pred_train = bag.predict(X_train)
y_pred_test = bag.predict(X_test)

In [15]:
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
print(f' Оценка r^2 на тренировочной выборке: \n{round(r2_train, 4)}')
print(f' Оценка r^2 на тестовой выборке: \n{round(r2_test, 4)}')

 Оценка r^2 на тренировочной выборке: 
0.9996
 Оценка r^2 на тестовой выборке: 
0.9963


In [16]:
msle_train = mean_squared_log_error(y_train, y_pred_train)
msle_test = mean_squared_log_error(y_test, y_pred_test)
print(f' Оценка MSLE на тренировочной выборке: \n{round(msle_train, 4)}')
print(f' Оценка MSLE на тестовой выборке: \n{round(msle_test, 4)}')

 Оценка MSLE на тренировочной выборке: 
0.0001
 Оценка MSLE на тестовой выборке: 
0.0007


In [17]:
rmsle_train = np.sqrt(mean_squared_log_error(y_train, y_pred_train))
rmsle_test = np.sqrt(mean_squared_log_error(y_test, y_pred_test))
print(f' Оценка RMSLE на тренировочной выборке: \n{round(rmsle_train, 4)}')
print(f' Оценка RMSLE на тестовой выборке: \n{round(rmsle_test, 4)}')

 Оценка RMSLE на тренировочной выборке: 
0.0082
 Оценка RMSLE на тестовой выборке: 
0.0269


In [18]:
mse_bag = mean_squared_error(y_test, y_pred_test)
print(f' Оценка MSE: \n{mse_bag}')
mae_bag = mean_absolute_error(y_test, y_pred_test)
print(f' Оценка MAE: \n{mae_bag}')

 Оценка MSE: 
9356.004990803905
 Оценка MAE: 
42.34655761341121


* **Вывод:** Оценки для этой модели улучшились относительно линейной регрессии, но относительно knn ухудшились: выросли MSE и MAE, на тестовой выборке выросло MSLE, и RMSLE

Таким образом, мы обучили модель прогнозирования цен на золото с параметрами: средняя цена за день индекса IMOEX, курс доллар рубль, ключевая ставка, фаза экономического роста. модель линейной регрессии показала плохой результат, что говорит о том, что цена золота не является линейно завивимой от определенных параметров, а имеет более сложное ценообразование. Моделью, которая показала минимальные ошибки, то есть наилучшей, является метод k ближайших соседей для регрессии с k = 2.